# Path C Jobs creation

In [7]:
import sys
import os
import json
#Personal Access token which will be replace by Service Token
os.environ['ASANA_ACCESS_TOKEN']=[Asana Token]
import asana
from six import print_
import re

#Project ID for "TEST AUTOMATION - LATAM Path C Position's Statuses & Sync"
project_id='1202074112575754'

In [4]:
#Gettin all the task for the specified project

if 'ASANA_ACCESS_TOKEN' in os.environ:
    client = asana.Client.access_token(os.environ['ASANA_ACCESS_TOKEN'])  
    tasks=client.tasks.find_by_project(project_id)
    tasks=[i for i in tasks]

In [5]:
#Getting all the Path C jobs available in BQ

from google.cloud import bigquery
bq_client=bigquery.Client()
query_output = bq_client.query(f"""select j.id, job_id,j.company, CONCAT('https://matching.turing.com/job/',j.id) as matching_link,
j.created_date, j.role, j.customer_category, j.opportunity_status, js.status as matching_status,
j.max_acceptable_rate,j.public_description
from devdb_mirror.ms2_job j
LEFT JOIN devdb_mirror.ms2_job_status js on js.id = j.job_status_id
inner JOIN devdb_mirror.ms2_job_type jt
on j.id = jt.job_id and jt.job_type_catalog_id = 4
where j.is_deleted = 0
""")
result_df = query_output.to_dataframe()
print("Total Path C jobs from BQ : "+str(len(result_df)))

/home/salman/Downloads/asana_venv/lib/python3.8/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/salman/Downloads/asana_venv/lib/python3.8/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more inform

Total Path C jobs from BQ : 123


In [8]:
#Search for ID's in task name

re_objects=[re.search('^\d{4}',task['name']) for task in tasks]
job_ids_already_created=[]
for re_object in re_objects:
    if re_object is not None:
        job_ids_already_created.append(int(re_object.group()))

In [9]:
#Filtering or Delta of new jobs

result_df=result_df[~result_df['id'].isin(job_ids_already_created)]
print("New path C jobs which do not exist before : "+str(len(result_df)))

New path C jobs which do not exist before : 81


In [ ]:
#Creating Asana Tasks from dataframe

for ind,row in result_df.iterrows():
    task_json={"memberships": [{"project": '1202074112575754', "section": '1202080202859365'}],
               'name': str(row['id'])+' - '+row['role'], 
                'notes': row['public_description'],
               'custom_fields':{"1202080200755485": row['matching_link'],
                                '1201752279340547': row['company'],
                               "1202080200763143": row['max_acceptable_rate']
                               }
          }
    client.tasks.create_in_workspace('1137034884867634', task_json)
    if(ind>1):
        break

# Devs assigned to Consultants

In [13]:
from google.cloud import bigquery
import re

dict_of_pipelines_with_gid={
'TEST AUTOMATION -  Personal Pipeline - Adrian':'1202110042372278',
'TEST AUTOMATION -  Personal Pipeline - Paz':'1202110042372319'} 

dict_of_consultant_name_to_pipelines={
'Adrian Olivera':'TEST AUTOMATION -  Personal Pipeline - Adrian',
'Paz Calderón':'TEST AUTOMATION -  Personal Pipeline - Paz'}

In [14]:
#Getting all the devs assigned to consultants from BQ

bq_client=bigquery.Client()
query_output = bq_client.query(f"""with last_updated_row as
(SELECT DISTINCT
dch.developer_id as dev_id, 
max(dch.time_changed) over(partition by dch.developer_id) as career_consultant_assigned_at,
nth_value(users.full_name,1) over(partition by dch.developer_id order by dch.time_changed desc)  as assigned_career_consultant
FROM devdb_mirror.developer_changes_history dch
LEFT JOIN devdb_mirror.tpm_user users ON users.id = cast(dch.new_value as int64)
WHERE dch.time_changed > '2021-06-30' and dch.column = "careerConsultantId" AND users.user_name IN
( SELECT email FROM custom_dashboards.sourcers_data))
Select lur.*,
dd.status,
CONCAT('https://matching.turing.com/developer/',lur.dev_id) as matching_link
from last_updated_row lur
inner join devdb_mirror.developer_detail dd on dd.user_id=lur.dev_id
""")
result_df = query_output.to_dataframe()
print(len(result_df))
result_df.head()

/home/salman/Downloads/asana_venv/lib/python3.8/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/salman/Downloads/asana_venv/lib/python3.8/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more inform

6678


,dev_id,career_consultant_assigned_at,assigned_career_consultant,status,matching_link
0,1890813,2022-03-24 19:23:39+00:00,Enrique Vazquez,passed ti,https://matching.turing.com/developer/1890813
1,506047,2021-12-10 18:58:54+00:00,Gabriel Chaves,failed,https://matching.turing.com/developer/506047
2,1741720,2021-12-22 16:59:45+00:00,Alan Vladimir Reyes Hernández,ofac restricted,https://matching.turing.com/developer/1741720
3,221344,2021-12-20 23:34:25+00:00,Alan Vladimir Reyes Hernández,ofac restricted,https://matching.turing.com/developer/221344
4,2155882,2022-03-31 14:55:29+00:00,Enrique Vazquez,additional mcq,https://matching.turing.com/developer/2155882


In [15]:
%%time
#Calculating delta of every consultant

client = asana.Client.access_token(os.environ['ASANA_ACCESS_TOKEN'])
for k,v in dict_of_consultant_name_to_pipelines.items():
    filtered_data=result_df[result_df['assigned_career_consultant']==k]
    sections=client.sections.find_by_project(dict_of_pipelines_with_gid[v])
    gid_of_pre_test=[item['gid'] for item in sections if item['name']=='Pre-Test'][0]
    tasks = client.tasks.get_tasks_for_section(gid_of_pre_test)
    list_of_devs_already_exist=[]
    gid_of_status=''
    gid_of_matching_link=''
    for task in tasks:
        task_info=client.tasks.get_task(task['gid'])
        for custom_field in task_info['custom_fields']:
            if custom_field['display_value']!=None and custom_field['display_value'].find('https://matching.turing.com/developer/')!=-1:
                list_of_devs_already_exist.append(int(re.search('\d+',custom_field['display_value']).group()))
                gid_of_matching_link=custom_field['gid']
            if custom_field['name']=='Status':
                gid_of_status=custom_field['gid']

    filtered_data=filtered_data[~filtered_data['dev_id'].isin(list_of_devs_already_exist)]
    print(k+' has '+str(len(filtered_data))+' new devs assigned')
    
    for ind,row in filtered_data.iterrows():
        task_json={"memberships": [{"project": dict_of_pipelines_with_gid[v], "section": gid_of_pre_test}],
               'name': str(row['dev_id']),
               'custom_fields':{gid_of_matching_link: row['matching_link'],
                               }
                  }
        client.tasks.create_in_workspace('1137034884867634', task_json)
        if(ind>0):
            break

/home/salman/Downloads/asana_venv/lib/python3.8/site-packages/asana/client.py:155: UserWarning: This request is affected by the "new_user_task_lists" deprecation. Please visit this url for more info: https://forum.asana.com/t/update-on-our-planned-api-changes-to-user-task-lists-a-k-a-my-tasks/103828
Adding "new_user_task_lists" to your "Asana-Enable" or "Asana-Disable" header will opt in/out to this deprecation and suppress this warning.
  warnings.warn(message)


Adrian Olivera has 29 new devs assigned


/home/salman/Downloads/asana_venv/lib/python3.8/site-packages/asana/client.py:155: UserWarning: This request is affected by the "new_project_templates" deprecation. Please visit this url for more info: https://forum.asana.com/t/a-new-api-for-project-templates/156432
Adding "new_project_templates" to your "Asana-Enable" or "Asana-Disable" header will opt in/out to this deprecation and suppress this warning.
  warnings.warn(message)


Paz Calderón has 254 new devs assigned
CPU times: user 350 ms, sys: 513 µs, total: 351 ms
Wall time: 1min 4s
